


<br>

# Objectif

L'objectif de ce travail était de tester la faisabilité d'harmoniser les 4 tables de RSA sur l'historique 2011 -- 2023 pour les intégrer dans un schéma homogène de données souhaité par Delta Lake (il est souhaité / préférable que le type, l'ordre et le nom des colonnes soient identiques pour faire du APPEND ; c'est en tout cas le cas avec `polars`).

- La première partie de ce document se concentre sur cette partie de normalisation des données (au sens où on les force à avoir toutes le même format d'année en année
- La 2ème partie teste l'intégration des données RSA de 2011 M12 à 2023 M04 sous forme de fichiers parquet écrit avec le module polars
- La 3ème partie comprend des tests de requêtage des données avec polars uniquement ; une fonction `remote_table('rsa@rsa')` permet d'accéder aux données via un `LazyFrame` polars flêchant vers les fichiers parquet
- La dernière partie montre le contenu d'un des fichiers json qui historise les étapes d'alimentations de la table `rsa@rsa` au format Delta

Au cours de ce test, un partitionnement par année-mois a été expérimenté mais les temps de requêtages étaient augmentés de manière importante (* 4, selon les volumes intégrés). L'option de mono-partition semble préférable tant que les volumes de données sont raisonnables.

Cette étape d'intégration en gardant toutes les colonnes des données PMSI sans en exclure aucune pourrait correspondre à l'étape de la couche bronze du lac de données, avec les données brutes associées.

N'ont pas été fait : 

- tester la commande SQL `ADD COLUMN` / `ALTER TABLE` sur certaines tables si les formats 2024 des données PMSI changeait, et pour les années à suivre.
- créer la clé primaire contenant l'année de sortie et le numéro de RSS accolés permettant d'avoir une unicité dans les relations
- tester la même chose sur les RSS, la même donnée contenant le lien avec les passages dans les unités médicales, mais cela peut être adapté de manière très similaire


<br>



In [1]:
import pandas as pd
import polars as pl
import pypmsi as pm

import pyspark
from delta import *
from delta.tables import *
from pyspark.sql.functions import *


# Intégrations

## Paramètres

In [2]:
p = pm.noyau_pmsi(finess = 290000017, 
              annee  = 2023,
              mois   = 12,
              path   = '/Users/guillaumepressiat/Documents/data/mco')

param_n_max = None


## Fonctions utiles

In [3]:
def normalized_schema():
    # Définir un schéma type de données

    # On récupère des années antérieures la dispersion des formats pmsi entre 2011 et 2023, 
    # cela permettra de mettre en cohérence les données sous un seul "schéma" avant de
    # les insérer en delta table

    rsa_temp = {'rsa' : pl.DataFrame(), 'actes' : pl.DataFrame(), 'diags' : pl.DataFrame(), 'rsa_um' : pl.DataFrame()}
    
    for i in range(2023, 2012, -1):
        # print(i)
        temp = p.irsa(annee = i, mois =  12, typi = 6, n_rows = 1000)
        
        # concat avec how = diagonal ici, on ajoute des colonnes si elles manquent
        rsa_temp['rsa'] = pl.concat([rsa_temp['rsa'], temp['rsa']], how="diagonal")
        rsa_temp['actes'] = pl.concat([rsa_temp['actes'], temp['actes']], how="diagonal")
        rsa_temp['diags'] = pl.concat([rsa_temp['diags'], temp['diags']], how="diagonal")
        rsa_temp['rsa_um'] = pl.concat([rsa_temp['rsa_um'], temp['rsa_um']], how="diagonal")

    temp = p.irsa(annee = 2023, mois =  12, typi = 6, n_rows = 1000)
    
    # pour aller vite, 2023 est à part car mois = 4
    rsa_temp['rsa'] = pl.concat([rsa_temp['rsa'], temp['rsa']], how="diagonal")
    rsa_temp['actes'] = pl.concat([rsa_temp['actes'], temp['actes']], how="diagonal")
    rsa_temp['diags'] = pl.concat([rsa_temp['diags'], temp['diags']], how="diagonal")
    rsa_temp['rsa_um'] = pl.concat([rsa_temp['rsa_um'], temp['rsa_um']], how="diagonal")
    
    rsa_temp['rsa'] = rsa_temp['rsa'].join(p.itra(annee = 2023, mois = 12, n_rows = 1000), how = 'left', on = 'cle_rsa')

    rsa_temp['rsa'] = (rsa_temp['rsa']
     .with_columns(pl.concat_str(['ansor', 'moissor']).alias('periode'),
                  pl.concat_str(['ansor', 'moissor']).alias('am')))
    
    empty = dict()
    empty['rsa'] = pl.DataFrame(schema = rsa_temp['rsa'].schema)
    empty['actes'] = pl.DataFrame(schema = rsa_temp['actes'].schema)
    empty['diags'] = pl.DataFrame(schema = rsa_temp['diags'].schema)
    empty['rsa_um'] = pl.DataFrame(schema = rsa_temp['rsa_um'].schema)

    return empty

def normalize_schema_rsa(dfd, table):
    # pour appliquer le schéma type de données si il n'est pas respecté

    # Sur la base des formats de toutes les années (unions permissives)
    # on concatène les rsa d'une année avec ces formats normalisés > on génère des colonnes null
    # pour avoir un schéma commun à toutes les années

    # how = align ici, et on ordonne les colonnes si elles ne sont pas dans le même ordre
    if table != 'rsa':
        df = (pl.concat([dfd[table],empty[table]], how = 'align')
              .select(sorted(empty[table].columns))
              .join(dfd['rsa'].select('cle_rsa', 'am', 'periode', 'nas', 
                                      'norss', 'ansor', 'moissor'), 
                    on = 'cle_rsa', how = 'inner').with_columns(
               pl.when(pl.col(pl.Utf8) == "")
            .then(None)
            .otherwise(pl.col(pl.Utf8))
            .name.keep()
        ))
    else:
        df = pl.concat([empty[table], dfd['rsa']], how = 'align').with_columns(
            pl.when(pl.col(pl.Utf8) == "")
            .then(None)
            .otherwise(pl.col(pl.Utf8)) # keep original value
            .name.keep()
        ).select(sorted(empty[table].columns))

    return df

# on prépare les rsa d'une année (on les normalise vis-à-vis du schéma unifié
# avec la fonction au dessus
# + aussi ajout du fichier tra et de deux colonnes de temps
def prepare_rsa(p, annee1, mois1, n_max = param_n_max):
    # Lire les données et les mettre sous le bon schéma

    
    rsa = p.irsa(typi = 6, annee = annee1, mois = mois1, n_rows = n_max)
    tra = p.itra(annee = annee1, mois = mois1, n_rows = n_max)
    rsa['rsa'] = rsa['rsa'].join(tra, on = 'cle_rsa', how = 'inner')
    rsa['rsa'] = (rsa['rsa']
                  .with_columns(pl.concat_str(['ansor', 'moissor']).alias('periode'),
                                pl.concat_str(['ansor', 'moissor']).alias('am')))
    rsa = {k: normalize_schema_rsa(rsa, k) for k, v in rsa.items()}

    return rsa


def one_delta(dfd, table):
    # pour écrire une des tables des rsa au format delta
    #print(dfd[table].schema)
    #print(dfd[table].shape)
    
    dfd[table].write_delta(
        "/Users/guillaumepressiat/Documents/data/delta/rsa/" + table, 
        #large_dtypes = True,
                    mode = "append") #, overwrite_schema = True


def write_rsa_delta(p, annee1, mois1):
    # pour lire les fichiers .rsa et tra, normaliser et écrire toutes les tables de rsa en delta
    
    temp = prepare_rsa(p, annee1, mois1)

    for k in ['rsa', 'actes', 'diags', 'rsa_um']:
        one_delta(temp, k)

def remote_table(pmsi_table):
    # pour faciliter la syntaxe, un wrapper pour accéder aux données LazyFrame
    
    return pl.scan_delta("/Users/guillaumepressiat/Documents/data/delta/" + pmsi_table.replace('@', '/'))

def count_pivot_periode(pmsi_table):
    # tri à plat dans les fichiers delta pour observer ce qui a été intégré

    tempd = remote_table(pmsi_table)
    r = (tempd
     .group_by('am', 'ansor', 'moissor')
     .len()
     .collect()
     .sort('moissor')
     .pivot('moissor', values = 'len', index = 'ansor',
            aggregate_function = 'sum', maintain_order=True)
     .sort('ansor')
    )
    return r



def delete_in_delta(pmsi_table, string_delete):
    # fonction pour supprimer des données dans delta table (utilise pyspark)
    
    dt_temp = DeltaTable.forPath(spark, '/Users/guillaumepressiat/Documents/data/delta/' + pmsi_table.replace('@', '/'))
    dt_temp.delete(string_delete)


## Chargement des données

In [4]:
empty = normalized_schema()
# empty['rsa'].schema

In [5]:
# de 2011 M12 à 2022 M12
for i in range(2013, 2024):
    # print(i)
    write_rsa_delta(p, i, 12)


In [6]:
# 2024 M07, année en cours, séparemment
# write_rsa_delta(p, 2024, 7)

In [7]:
# remote_table('rsa@actes').collect().filter(pl.col('ansor').is_null())

## Volumétries

In [8]:
%%time
count_pivot_periode('rsa@rsa')

CPU times: user 136 ms, sys: 30.3 ms, total: 166 ms
Wall time: 54.3 ms


ansor,01,02,03,04,05,06,07,08,09,10,11,12
str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
"""2013""",9167,8806,9540,9158,8465,8679,9317,8032,8305,9309,8432,8751
"""2014""",9405,8525,9006,9063,8563,8986,9274,8263,9450,9926,8779,9518
"""2015""",9296,8970,9763,9019,8374,9561,9216,8171,9493,9999,9419,9905
"""2016""",9724,9591,10515,9912,9812,10216,9229,9272,10269,10187,10333,10567
"""2017""",10668,9656,11052,9818,10936,11431,10230,10274,10662,11118,10731,10361
…,…,…,…,…,…,…,…,…,…,…,…,…
"""2019""",11665,10526,11687,11162,10776,10660,11233,9841,10745,11357,10570,11238
"""2020""",11885,10942,10220,8568,9483,11289,11469,10046,11786,11417,10480,11335
"""2021""",11310,10858,12453,11879,11453,12084,11418,10650,12091,11910,11593,11981


In [9]:
%%time
count_pivot_periode('rsa@rsa_um')

CPU times: user 139 ms, sys: 23.7 ms, total: 163 ms
Wall time: 50.6 ms


ansor,01,02,03,04,05,06,07,08,09,10,11,12
str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
"""2013""",10330,9828,10647,10439,9549,9711,10512,8966,9341,10474,9427,10017
"""2014""",10624,9735,10185,10269,9633,10124,10454,9283,10494,11090,9982,10797
"""2015""",10478,10145,11098,10246,9490,10877,10511,9181,10668,11344,10679,11236
"""2016""",10841,10739,11880,11112,11013,11448,10382,10437,11431,11295,11570,11891
"""2017""",11940,10976,12389,10960,12232,12629,11463,11320,11811,12396,11968,11636
…,…,…,…,…,…,…,…,…,…,…,…,…
"""2019""",13112,11791,13112,12476,12075,11918,12519,10909,12017,12720,11853,12524
"""2020""",13209,12312,11564,9666,10632,12422,12671,11007,13037,12627,11629,12613
"""2021""",12480,12037,13841,13153,12636,13353,12683,11718,13282,13238,12878,13286


In [10]:
%%time
count_pivot_periode('rsa@actes')

CPU times: user 407 ms, sys: 66.7 ms, total: 473 ms
Wall time: 133 ms


ansor,01,02,03,04,05,06,07,08,09,10,11,12
str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
"""2013""",39450,35718,40894,38241,34470,36537,39552,35240,34559,37183,33334,36499
"""2014""",36827,36179,36302,37509,33753,33814,34192,31689,32734,29792,31938,35835
"""2015""",34416,33117,37165,34225,33908,35155,36629,30097,35562,36297,34807,36216
"""2016""",34992,35461,39169,36779,36655,36178,34188,33633,36894,35207,35265,36268
"""2017""",38756,35172,38395,35454,35330,37909,38250,34571,36344,36986,34595,36148
…,…,…,…,…,…,…,…,…,…,…,…,…
"""2019""",39926,35786,40429,37587,35955,36130,37412,31360,37032,37933,33767,35833
"""2020""",37193,35813,34328,27193,30666,35311,37232,33115,37461,37052,33694,34086
"""2021""",34487,35112,41197,37287,33438,36341,35283,31555,34607,33530,31714,34134


In [11]:
%%time
count_pivot_periode('rsa@diags')

CPU times: user 748 ms, sys: 183 ms, total: 931 ms
Wall time: 259 ms


ansor,01,02,03,04,05,06,07,08,09,10,11,12
str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
"""2013""",51552,48464,55267,56511,50818,51976,56642,48546,51040,57132,50241,53880
"""2014""",58166,53540,55703,57256,52912,54255,55540,50383,55682,59274,52908,56873
"""2015""",55375,55018,59977,56956,50693,59464,57639,50895,57099,60067,56522,60209
"""2016""",59527,59878,66432,62540,60556,62283,57176,57516,62489,61596,62967,63929
"""2017""",65743,61249,68527,60981,66991,67664,63843,62677,62332,65690,65671,64233
…,…,…,…,…,…,…,…,…,…,…,…,…
"""2019""",72868,67123,73669,70692,65988,66030,71534,61964,67261,68819,62005,65833
"""2020""",70200,64916,62578,55956,59858,67101,68166,60045,68677,68823,62851,66577
"""2021""",67093,65017,74707,70623,67517,68375,69498,64400,69719,69529,67911,67933


## Update (delete de l'année en cours)

On passe à pyspark pour pouvoir lancer des delete dans les fichiers delta lake
(il faut installer un java)


```python
# .noeval 

from pyspark.sql import SparkSession

builder = pyspark.sql.SparkSession.builder.appName("MyPmsiDLake") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

# https://stackoverflow.com/a/24657630/10527496
# https://stackoverflow.com/questions/70192980/error-creating-sparkcontext-locally-an-error-occurred-while-calling-none-org-apa
```



In [12]:
annee_delete = '2023'
string_delete = "ansor = \"" + annee_delete + "\""
string_delete
# string_delete = "ansor is NULL"

'ansor = "2023"'

```python
#.noeval

for i in ['rsa@rsa', 'rsa@actes', 'rsa@diags', 'rsa@rsa_um']:
    delete_in_delta(i, string_delete)
```



2023 a bien été supprimé

## vacuum, à faire

Nettoyer régulièrement les fichiers parquet inutiles

# Tests

On ne peut pas joindre les tables delta avec polars > on scan / filtre et on collecte les éléments par morceaux.

Par morceaux, on entend : entre parenthèses, on collecte les lignes correspondants à la table `rsa@rsa`, puis la table `rsa@actes` et on joint ensuite ces deux tables.

Dans notre exemple ici : 

- on s'intéresse aux séjours-RSS chirurgicaux (GHM type C), sortis en mai 2020
- qui ont un acte de remplacement de l'articulation hanche ou genou par prothèse totale (actes N.KA), on filtre sur l'activité CCAM 1
- on rajoutera une clause ensuite sur la présence d'un diagnostic S72, fracture du col du fémur en position principale du séjour (position = 1)

<br>

::: {.panel-tabset}


## rsa

``` {.python}
# d'abord les rsa concernés par le filtre GHM C sortis en mai 2020
(remote_table('rsa@rsa')
 .filter(pl.col('am') == '202005')
 .filter(pl.col('rsatype') == 'C')
 .unique()
 .collect()
)
```

## actes

``` {.python}
# Ensuite les actes correspondants au pattern
(remote_table('rsa@actes')
 .filter((pl.col('am') == '202005')) 
 .filter(pl.col('cdccam').str.contains('N.KA'))
 .filter(pl.col('act') == '1')
 .unique()
 .collect()
)
```

## diagnostics

``` {.python}
# Et la table des diagnostics
(remote_table('rsa@diags')
 .filter((pl.col('am') == '202005')) 
 .filter(pl.col('diag').str.contains('S72'))
 .filter(pl.col('position') == 1)
 .select('cle_rsa')
 .unique()
 .collect()
)
```

:::


## rsa partie fixe & actes

In [13]:
%%time
# Jointure entre la table des RSA partie fixe et la table des actes
(remote_table('rsa@rsa')
 .filter(pl.col('am') == '202005')
 .filter(pl.col('rsatype') == 'C')
 .unique()
 .collect()
).join(
(remote_table('rsa@actes')
 .filter((pl.col('am') == '202005')) 
 .filter(pl.col('cdccam').str.contains('N.KA'))
 .filter(pl.col('act') == '1')
 .unique()
 .collect()
), on = 'cle_rsa', how = 'inner'
).group_by('nofiness', 'am').agg(pl.col('cle_rsa').n_unique().alias('count'))

CPU times: user 1.63 s, sys: 327 ms, total: 1.96 s
Wall time: 629 ms


nofiness,am,count
str,str,u32
"""290000017""","""202005""",32


En alternative, on peut aussi utiliser les colonnes "stream" de la table RSA qui contiennent tous les actes, diags et permettent d'éviter de faire des jointures entre plusieurs tables quand l'information souhaitée est juste (présence ou absence de l'acte, du diag, etc.), c'est ce qui est fait ci-dessous

In [14]:
%%time
# alternative en utilisant la zone "stream" des actes en ligne
(remote_table('rsa@rsa')
 .filter(
     (pl.col('am') == '202005') & 
     (pl.col('rsatype') == 'C') & 
         (pl.col('stream_actes').str.contains('N.KA')))
 .unique()
 .collect()
).group_by('nofiness', 'am').agg(pl.col('cle_rsa').n_unique().alias('count'))

CPU times: user 1.82 s, sys: 294 ms, total: 2.11 s
Wall time: 685 ms


nofiness,am,count
str,str,u32
"""290000017""","""202005""",32


## rsa partie fixe & actes & diags

On ajoute la clause sur le diagnostic principal S72

<br>


In [15]:
%%time
# Jointure entre la table des RSA partie fixe et la table des actes
(remote_table('rsa@rsa')
 .filter(pl.col('am') == '202005')
 .filter(pl.col('rsatype') == 'C')
 .unique()
 .collect()
).join(
(remote_table('rsa@actes')
 .filter((pl.col('am') == '202005')) 
 .filter(pl.col('cdccam').str.contains('N.KA'))
 .filter(pl.col('act') == '1')
 .unique()
 .collect()
), on = 'cle_rsa', how = 'inner'
).join(
(remote_table('rsa@diags')
 .filter((pl.col('am') == '202005')) 
 .filter(pl.col('diag').str.contains('S72'))
 .filter(pl.col('position') == 1)
 .select('cle_rsa')
 .unique()
 .collect()
), on = 'cle_rsa', how = 'inner'
).group_by('nofiness', 'am').agg(pl.col('cle_rsa').n_unique().alias('count'))

CPU times: user 2.16 s, sys: 271 ms, total: 2.43 s
Wall time: 861 ms


nofiness,am,count
str,str,u32
"""290000017""","""202005""",14


In [16]:
%%time
# alternative en utilisant la zone "stream" des actes en ligne, et la colonne dp
(remote_table('rsa@rsa')
 .filter(
     (pl.col('am') == '202005') & 
     (pl.col('rsatype') == 'C') & 
         (pl.col('stream_actes').str.contains('N.KA')) & 
         (pl.col('dp').str.contains('S72')))
 .unique()
 .collect()
).group_by('nofiness', 'am').len()

CPU times: user 1.77 s, sys: 214 ms, total: 1.98 s
Wall time: 657 ms


nofiness,am,len
str,str,u32
"""290000017""","""202005""",14


## Nombre de lignes / colonnes des tables


**Après avoir enlevé 2023 M04**

In [17]:
remote_table('rsa@rsa').collect().shape

(1386059, 108)

In [18]:
remote_table('rsa@actes').collect().shape

(4705532, 19)

In [19]:
remote_table('rsa@diags').collect().shape

(8394120, 10)

In [20]:
remote_table('rsa@rsa_um').collect().shape

(1547380, 23)

# Description des rsa

Avec la fonction `describle` de `polars` on affiche des statistiques surles données rsa.

Le choix a été fait d'ordonner les colonnes par ordre alphabétique.


In [21]:
remote_table('rsa@rsa').collect().describe()

statistic,admission_maison_naissance,adnp75,agean,agegest,agejr,am,anivgprec,anseqta,ansor,cat_nb_intervenants,cdgeo,cdpostal,cdpu,cle_rsa,confcdsej,conversion_hc,delaireg,dest,dp,dr,dtent,dtsort,duree,echpmsi,ell_forf_diabete,ell_gradation,genautorsa,ghm,ghm1,ghshorsinno,ghsminore,gpcdretr,gpcmd,gpcompx,gpnum,gptype,…,novrsa,novrss,numinno,paslitsp,pc_raac,periode,poids,prov,rescrit_tarifaire,resererve_hosp,rsacdretr,rsacmd,rsacompx,rsanum,rsatype,rsavclass,schpmsi,sejinfbi,sexe,stream_actes,stream_das,stream_dpum,stream_drum,stream_um,suppdefcard,surveillance_particuliere,topadmnais,topctc,topradalimta,topradavastin,typedosim,typesej,typmachradio,typrestpo,uhcd,valvaort,zrdth
str,str,str,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""count""","""553964""","""1658""",1.3391e6,52416.0,46959.0,"""1386059""","""119631""","""1386059""","""1386059""","""20251""","""1386059""","""553964""","""19651""","""1386059""","""1458""","""899""",42751.0,"""94975""","""1386059""","""670930""","""1386059""","""1386059""",1.386059e6,"""1386059""","""109269""","""337271""","""1171340""","""1386059""","""1386059""","""1060154""","""1153074""","""1386059""","""1386059""","""1386059""","""1386059""","""1386059""",…,"""1386059""","""1386059""","""7""","""1386059""","""663233""","""1386059""",30167.0,"""296197""","""2""","""6953""","""1386059""","""1386059""","""1386059""","""1386059""","""1386059""","""1386059""","""1386059""","""1386059""","""1386059""","""1058106""","""707944""","""1386059""","""677458""","""1386059""","""920203""","""20007""","""143463""","""143463""","""0""","""1040839""","""259236""","""1720""","""209464""","""1386059""","""1386059""","""2057""","""7306"""
"""null_count""","""832095""","""1384401""",46959.0,1.333643e6,1.3391e6,"""0""","""1266428""","""0""","""0""","""1365808""","""0""","""832095""","""1366408""","""0""","""1384601""","""1385160""",1.343308e6,"""1291084""","""0""","""715129""","""0""","""0""",0.0,"""0""","""1276790""","""1048788""","""214719""","""0""","""0""","""325905""","""232985""","""0""","""0""","""0""","""0""","""0""",…,"""0""","""0""","""1386052""","""0""","""722826""","""0""",1.355892e6,"""1089862""","""1386057""","""1379106""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""327953""","""678115""","""0""","""708601""","""0""","""465856""","""1366052""","""1242596""","""1242596""","""1386059""","""345220""","""1126823""","""1384339""","""1176595""","""0""","""0""","""1384002""","""1378753"""
"""mean""",null,null,58.715123,38.508795,57.895143,null,null,null,null,null,null,null,null,null,null,null,214.421207,null,null,null,"""2018-10-20 05:45:58.617000""","""2018-10-22 18:00:11.703000""",2.509874,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,3181.599463,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,null,21.579356,2.805596,95.845139,null,null,null,null,null,null,null,null,null,null,null,87.009044,null,null,null,null,null,6.340697,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,709.952416,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""min""","""0""","""1""",1.0,17.0,0.0,"""201301""","""0""","""2012""","""2013""","""A""","""1000""","""1090""","""5""","""1""","""1""","""1""",0.0,"""1""","""A010""","""A010""","""2012-04-23""","""2013-01-01""",0.0,"""0""","""0""","""0""","""0""","""01C031""","""01C031""","""22""","""0""","""0""","""1""","""1""","""0""","""C""",…,"""220""","""116""","""INNOV1441001C""","""0""","""0""","""201301"""

# Historique

On peut retracer les types d'actions réalisées avec les fichiers de méta-données json.

In [22]:
from deltalake import DeltaTable
dt = DeltaTable("/Users/guillaumepressiat/Documents/data/delta/rsa/rsa")
dt.history()

[{'timestamp': 1725821403775,
  'operation': 'WRITE',
  'operationParameters': {'mode': 'Append'},
  'clientVersion': 'delta-rs.0.19.1',
  'operationMetrics': {'execution_time_ms': 266,
   'num_added_files': 1,
   'num_added_rows': 143463,
   'num_partitions': 0,
   'num_removed_files': 0},
  'version': 10},
 {'timestamp': 1725821376560,
  'operation': 'WRITE',
  'operationParameters': {'mode': 'Append'},
  'operationMetrics': {'execution_time_ms': 256,
   'num_added_files': 1,
   'num_added_rows': 141901,
   'num_partitions': 0,
   'num_removed_files': 0},
  'clientVersion': 'delta-rs.0.19.1',
  'version': 9},
 {'timestamp': 1725821349348,
  'operation': 'WRITE',
  'operationParameters': {'mode': 'Append'},
  'clientVersion': 'delta-rs.0.19.1',
  'operationMetrics': {'execution_time_ms': 259,
   'num_added_files': 1,
   'num_added_rows': 139680,
   'num_partitions': 0,
   'num_removed_files': 0},
  'version': 8},
 {'timestamp': 1725821322761,
  'operation': 'WRITE',
  'operationParame